In [1]:
from sagemaker import get_execution_role
import boto3
import sagemaker

In [3]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
bucket = 'mysagemakertutorial'
data_prefix = "sagemaker/data"
model_prefix = "sagemaker/model"
sess = sagemaker.Session()

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "latest")
xgboost_container

'433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest'

In [4]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train/train.csv'.format(bucket, data_prefix), content_type='csv')
s3_input_train

In [6]:
xgb = sagemaker.estimator.Estimator(xgboost_container,role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/'.format(bucket, model_prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        num_round=10)

In [7]:
%%time
xgb.fit({'train': s3_input_train})

2021-12-22 16:10:49 Starting - Starting the training job...
2021-12-22 16:11:12 Starting - Launching requested ML instancesProfilerReport-1640189449: InProgress
......
2021-12-22 16:12:12 Starting - Preparing the instances for training.........
2021-12-22 16:13:39 Downloading - Downloading input data...
2021-12-22 16:14:19 Training - Training image download completed. Training in progress..Arguments: train
[2021-12-22:16:14:20:INFO] Running standalone xgboost training.
[2021-12-22:16:14:20:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-12-22:16:14:20:INFO] File size need to be processed in the node: 0.1mb. Available memory size in the node: 8363.38mb
[2021-12-22:16:14:20:INFO] Determined delimiter of CSV input is ','
[16:14:20] S3DistributionType set as FullyReplicated
[16:14:20] 453x30 matrix with 13590 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[16:14:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 12